# **log**

In [1]:
'''from google.colab import drive
drive.mount("/content/gdrive")'''

'from google.colab import drive\ndrive.mount("/content/gdrive")'

# **Prepare**

In [2]:
!pip install transformers
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install nltk
!pip install cuda-python
!pip install Keras-Preprocessing
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import math
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import BertTokenizer
from transformers import DistilBertTokenizer, RobertaTokenizer
from tensorflow.keras.optimizers import Adam, SGD
from transformers import TFXLNetModel, XLNetTokenizer
import nltk
import re
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from numpy import array
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from tensorflow.keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers import Concatenate
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from tqdm import tqdm, trange


Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
ERROR: Could not find a version that satisfies t

/project/6001568/zhe8nov1/jp1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-13 20:19:53.901420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 20:19:58.428904: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-13 20:20:31.508554: W tensorflow/compiler/xla/stream_executor/platform/d

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:

train = pd.read_csv("train.scruples-anecdotes.csv" ,skip_blank_lines=True)

dev =pd.read_csv("dev-scruples-anecdotes.csv", skip_blank_lines=True)


test =pd.read_csv("test.scruples-anecdotes.csv", skip_blank_lines=True)



In [5]:
print(train['text'].isnull().sum())
print(dev['text'].isnull().sum())
print(test['text'].isnull().sum())

22
1
4


In [6]:
train = train.dropna()
dev = dev.dropna()
test = test.dropna()

In [7]:
print(train['text'].isnull().sum())
print(dev['text'].isnull().sum())
print(test['text'].isnull().sum())

0
0
0


In [8]:
train.shape

(26196, 10)

In [9]:
train

,id,post_id,action,title,text,post_type,label_scores,label,binarized_label_scores,binarized_label
0,yqk9ZSYu3Vd9A6sUBIGPLJ0mTCxcsv74,ar2iqw,"{'description': 'hiding my controller', 'prono...",AITA for hiding my controller?,"Backstory: So, I got an Xbox one for Christmas...",HISTORICAL,"{'AUTHOR': 0, 'OTHER': 22, 'EVERYBODY': 0, 'NO...",4,"{'RIGHT': 22, 'WRONG': 0}",RIGHT
1,xBCLN7LVJBGNAakxDVBy0fOrtpfTE577,aqtrol,"{'description': 'ratting out my supervisor', '...",AITA For Ratting Out My Supervisor?,I work with about six other people at might jo...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 5, 'EVERYBODY': 0, 'NOB...",4,"{'RIGHT': 5, 'WRONG': 0}",RIGHT
2,IdwLUU6ucUJfK8pL1Uv82dQpPVvgppO8,b01ajb,{'description': 'commenting that unfamiliar in...,AITA for commenting that unfamiliar indie band...,Context: There was an Instagram post about unp...,HISTORICAL,"{'AUTHOR': 3, 'OTHER': 4, 'EVERYBODY': 0, 'NOB...",4,"{'RIGHT': 5, 'WRONG': 3}",RIGHT
3,pbFtHLhjFcEF0MpceVPvk4oGmQeKof8D,a4m4zq,{'description': 'arguing for my friend to surr...,AITA for arguing for my Friend to Surrender in...,Me and my friends spent sometime organizing a ...,HISTORICAL,"{'AUTHOR': 5, 'OTHER': 0, 'EVERYBODY': 0, 'NOB...",0,"{'RIGHT': 0, 'WRONG': 5}",WRONG
4,IZzCnvsXj8vBAhArM0gxgALAwrd50yHG,apna7l,{'description': 'being mad at my wife for negl...,AITA for being mad at my wife for neglecting/ ...,A little background. I'm a far from rich guy w...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 5, 'EVERYBODY': 4, 'NOB...",4,"{'RIGHT': 5, 'WRONG': 4}",RIGHT
...,...,...,...,...,...,...,...,...,...,...
27761,MsxMg8eUAk7L2DuE1AT4IlwMzXZ90rUP,amjxh1,{'description': 'getting mad at my girlfriend ...,AITA for getting mad at my girlfriend for dism...,So a bit of background info. My girlfriend has...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 1, 'EVERYBODY': 1, 'NOB...",4,"{'RIGHT': 1, 'WRONG': 1}",RIGHT
27762,OYJdH0Swn7PxoOD3XfKdOtotoHLgMRvN,amuw37,"{'description': ""feeling shitty my best friend...",AITA for feeling shitty my best friend didn’t ...,Context:\n\nI’ve been best friends with this g...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 1, 'EVERYBODY': 0, 'NOB...",4,"{'RIGHT': 2, 'WRONG': 0}",RIGHT
27763,XOamt7UoD9nOsnQjNUWfsKx4jdGV708h,ajta6b,{'description': 'commenting on how my gf and h...,AITA for commenting on how my gf and her mum a...,So me (19M) and my gf’s (18F) relationship has...,HISTORICAL,"{'AUTHOR': 1, 'OTHER': 1, 'EVERYBODY': 0, 'NOB...",0,"{'RIGHT': 2, 'WRONG': 1}",RIGHT
27764,FJ2u1Tojs9VtXH52bb1iLBAdP6WYn3Yt,b0cdwc,{'description': 'missing out on family events ...,AITA for missing out on family events to work ...,"A little info, I’m an Early College student, f...",HISTORICAL,"{'AUTHOR': 1, 'OTHER': 1, 'EVERYBODY': 0, 'NOB...",3,"{'RIGHT': 3, 'WRONG': 1}",RIGHT


In [10]:
train_df = train[['text','binarized_label']]

In [11]:
dev_df = dev[['text','binarized_label']]

In [12]:
test_df =  test[['text','binarized_label']]

In [13]:
dev_df

,text,binarized_label
0,My boyfriend (well just call him M) and I are ...,RIGHT
2,\n\nMy really good friend is getting married....,RIGHT
3,"I've been very lonely for a long time, few fri...",RIGHT
4,Alt account because friends know my real one. ...,RIGHT
5,Pretty much was hanging out with two friends. ...,WRONG
...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG
2495,So this just happened and I'm a little taken a...,WRONG
2497,"So, I've been friends with someone for a few y...",RIGHT
2498,My first post!\n\nNo words were exchanged in t...,RIGHT


In [14]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

label_encoder = preprocessing.LabelEncoder()

train_df['num_binarized_label']= label_encoder.fit_transform(train_df['binarized_label'])
dev_df['num_binarized_label']= label_encoder.fit_transform(dev_df['binarized_label'])
test_df['num_binarized_label']= label_encoder.fit_transform(test_df['binarized_label'])


In [15]:
train_df

,text,binarized_label,num_binarized_label
0,"Backstory: So, I got an Xbox one for Christmas...",RIGHT,0
1,I work with about six other people at might jo...,RIGHT,0
2,Context: There was an Instagram post about unp...,RIGHT,0
3,Me and my friends spent sometime organizing a ...,WRONG,1
4,A little background. I'm a far from rich guy w...,RIGHT,0
...,...,...,...
27761,So a bit of background info. My girlfriend has...,RIGHT,0
27762,Context:\n\nI’ve been best friends with this g...,RIGHT,0
27763,So me (19M) and my gf’s (18F) relationship has...,RIGHT,0
27764,"A little info, I’m an Early College student, f...",RIGHT,0


In [16]:
nbl1 = train_df['num_binarized_label'] == 1
train_df_1 = pd.DataFrame(train_df[nbl1])
train_df_1

,text,binarized_label,num_binarized_label
3,Me and my friends spent sometime organizing a ...,WRONG,1
18,A girl I dated a few years ago and I have rema...,WRONG,1
22,TLDR at bottom.\n\nSo went to a nice dinner wi...,WRONG,1
27,"For some context, I live in a flatshare with t...",WRONG,1
30,I (24M) have been with my girlfriend(29F) for ...,WRONG,1
...,...,...,...
27742,It all started when I gave her the password to...,WRONG,1
27743,"Bit of a backstory, my boyfriend and I began d...",WRONG,1
27747,I work in a very small team at my company. So ...,WRONG,1
27751,"On Valentine’s Day, my GF and I had a great di...",WRONG,1


In [17]:
nbl0 = train_df['num_binarized_label'] == 0
train_df_new = pd.DataFrame(train_df[nbl0]) # dataframe that only contains '0' as num_binarized_label
#train_df_new
train_df_0 = train_df_new.iloc[:5738]
#train_df_0

In [18]:
balance_train_df = train_df_0.append(train_df_1, ignore_index=True)
#balance_train_df
train_df = balance_train_df.sample(frac = 1) # shuffle

In [19]:
train_df

,text,binarized_label,num_binarized_label
2901,I work remotely while also studying in college...,RIGHT,0
10635,"I (29) am severely visually impaired, to the p...",WRONG,1
3256,For the past 6 months my dad has been really c...,RIGHT,0
3131,A little background. My wife and I are of diff...,RIGHT,0
4198,To sum up the entire story as quickly as possi...,RIGHT,0
...,...,...,...
6613,"So, to clarify, my sister and i have a fairly ...",WRONG,1
7055,My ex girlfriend and I broke up at the beginni...,WRONG,1
7200,So the title is a very condensed version of wh...,WRONG,1
200,"Several years ago when I was in college, my ol...",RIGHT,0


In [20]:
def clean_text(text):
    #Remove emojis and special chars
    clean=text
    #reg = re.compile('\\.+?(?=\B|$)')
    #clean = text.apply(lambda r: re.sub(reg, string=r, repl=''))
    #reg = re.compile('\x89Û_')
    #clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    reg = re.compile('\&amp')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='&'))
    reg = re.compile('\\n')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    
    #Remove hashtag symbol (#)
    #clean = clean.apply(lambda r: r.replace('#', ''))

    #Remove user names
    reg = re.compile('@[a-zA-Z0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='@'))

    #Remove URLs
    reg = re.compile('https?\S+(?=\s|$)')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='www'))

    #Lowercase
    #clean = clean.apply(lambda r: r.lower())
    return clean

In [21]:
train_df = train_df.replace(np.nan, '', regex=True)

dev_df = dev_df.replace(np.nan, '', regex=True)

test_df = test_df.replace(np.nan, '', regex=True)


In [22]:
train_df['clean'] = clean_text(train_df['text'])
dev_df['clean'] = clean_text(dev_df['text'])
test_df['clean'] = clean_text(test_df['text'])

In [23]:
train_df

,text,binarized_label,num_binarized_label,clean
2901,I work remotely while also studying in college...,RIGHT,0,I work remotely while also studying in college...
10635,"I (29) am severely visually impaired, to the p...",WRONG,1,"I (29) am severely visually impaired, to the p..."
3256,For the past 6 months my dad has been really c...,RIGHT,0,For the past 6 months my dad has been really c...
3131,A little background. My wife and I are of diff...,RIGHT,0,A little background. My wife and I are of diff...
4198,To sum up the entire story as quickly as possi...,RIGHT,0,To sum up the entire story as quickly as possi...
...,...,...,...,...
6613,"So, to clarify, my sister and i have a fairly ...",WRONG,1,"So, to clarify, my sister and i have a fairly ..."
7055,My ex girlfriend and I broke up at the beginni...,WRONG,1,My ex girlfriend and I broke up at the beginni...
7200,So the title is a very condensed version of wh...,WRONG,1,So the title is a very condensed version of wh...
200,"Several years ago when I was in college, my ol...",RIGHT,0,"Several years ago when I was in college, my ol..."


In [24]:
test_df['clean'][0]

"   Throwaway for obvious reasons.     I dropped out of uni 2 years ago because I was planning on moving to a different country and going to Uni there. However, my parents didn't want to help me in any way and I couldn't afford it myself. They've always supported my younger sister though, bought her an apartment and they're even paying her bills, even though she earns enough money to do that herself. So atm I'm stuck at home with them, which sucks as I am 25 yo and I feel like I'm wasting my life completely.     Both my parents were extremely abusive, especially my dad. As a result, I have social anxiety, trust issues, panic attacks etc. I can deal with everything as I got used to it anyway, but lately things have gotten worse.     A few years ago my dad was diagnosed with type II diabetes, and he retired. He's been staying at home ever since and he's slowly but surely becoming a vegetable. He had a stroke which left him unable to speak/walk properly. I'm no doctor, but I found his hos

# **Data preprocessing**

In [25]:
train_df["lower"] = train_df["clean"].apply(lambda x: x.lower())

dev_df["lower"] = dev_df["clean"].apply(lambda x: x.lower())

test_df["lower"] = test_df["clean"].apply(lambda x: x.lower())
display(train_df.head())

,text,binarized_label,num_binarized_label,clean,lower
2901,I work remotely while also studying in college...,RIGHT,0,I work remotely while also studying in college...,i work remotely while also studying in college...
10635,"I (29) am severely visually impaired, to the p...",WRONG,1,"I (29) am severely visually impaired, to the p...","i (29) am severely visually impaired, to the p..."
3256,For the past 6 months my dad has been really c...,RIGHT,0,For the past 6 months my dad has been really c...,for the past 6 months my dad has been really c...
3131,A little background. My wife and I are of diff...,RIGHT,0,A little background. My wife and I are of diff...,a little background. my wife and i are of diff...
4198,To sum up the entire story as quickly as possi...,RIGHT,0,To sum up the entire story as quickly as possi...,to sum up the entire story as quickly as possi...


In [26]:
!pip install contractions
import contractions
train_df["lower"] = train_df["lower"].apply(lambda x: contractions.fix(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: contractions.fix(x))
test_df["lower"] = test_df["lower"].apply(lambda x: contractions.fix(x))

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [27]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text) # or ''.join([x for x in text if x in string.printable]) 

# remove non-ascii characters from the text
train_df["lower"] = train_df["lower"].apply(lambda x: remove_non_ascii(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: remove_non_ascii(x))
test_df["lower"] = test_df["lower"].apply(lambda x: remove_non_ascii(x))

In [28]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# remove non-ascii characters from the text
train_df["lower"] = train_df["lower"].apply(lambda x: remove_special_characters(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: remove_special_characters(x))
test_df["lower"] = test_df["lower"].apply(lambda x: remove_special_characters(x))

In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/zhe8nov1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
# Tokenizing the tweet base texts.
from nltk.tokenize import word_tokenize
train_df['tokenized'] = train_df['lower'].apply(word_tokenize)
dev_df['tokenized'] = dev_df['lower'].apply(word_tokenize)
test_df['tokenized'] = test_df['lower'].apply(word_tokenize)

In [31]:
from nltk.stem import SnowballStemmer

def snowball_stemmer(text):
    """
        Stem words in list of tokenized words with SnowballStemmer
    """
    stemmer = nltk.SnowballStemmer("english")
    stems = [stemmer.stem(i) for i in text]
    return stems


In [32]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/zhe8nov1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zhe8nov1/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [33]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma



lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_wo_pos'] = train_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

dev_df['lemmatize_word_wo_pos'] = dev_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

test_df['lemmatize_word_wo_pos'] = test_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [34]:
connect1 = []
for i in train_df['lemmatize_word_wo_pos']:
  connect1.append(' '.join(i))
train_df['clean1'] = connect1


connect2 = []
for i in dev_df['lemmatize_word_wo_pos']:
  connect2.append(' '.join(i))
dev_df['clean1'] = connect2


connect3 = []
for i in test_df['lemmatize_word_wo_pos']:
  connect3.append(' '.join(i))
test_df['clean1'] = connect3


In [35]:
dev_df['clean'][0]

'My boyfriend (well just call him M) and I are creators on Instagram (he is also on TikTok). I\'m a digital and traditional artist and in one of my classes we create t shirts and things (socks, lanyards, water bottles, stickers, you name it). I\'m a big fan of putting my art on t shirts so I can sell them.  M wanted to make some posts for TikTok and wanted to use the shirt I made today. My one personal rule is that if the shirt isn\'t being given to someone else, then I wear it first (this is because I know I wont get it back. Ive made one other shirt that I\'ve worn, and he wore it for a week and hasn\'t given it back.) So I told him no, because I haven\'t worn the shirt yet.   M proceeded to complain about not having content to post, and how his IG post today didn\'t get any engagement, saying "of course you dont know, because you never look."  Am I the asshole for wanting to wear the shirt I made just once?'

In [36]:
dev_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1
0,My boyfriend (well just call him M) and I are ...,RIGHT,0,My boyfriend (well just call him M) and I are ...,my boyfriend (well just call him m) and i are ...,"[my, boyfriend, (, well, just, call, him, m, )...","[my, boyfriend, (, well, just, call, him, m, )...",my boyfriend ( well just call him m ) and i ar...
2,\n\nMy really good friend is getting married....,RIGHT,0,My really good friend is getting married. Y...,my really good friend is getting married. y...,"[my, really, good, friend, is, getting, marrie...","[my, really, good, friend, is, getting, marrie...",my really good friend is getting married . yay...
3,"I've been very lonely for a long time, few fri...",RIGHT,0,"I've been very lonely for a long time, few fri...","i have been very lonely for a long time, few f...","[i, have, been, very, lonely, for, a, long, ti...","[i, have, been, very, lonely, for, a, long, ti...","i have been very lonely for a long time , few ..."
4,Alt account because friends know my real one. ...,RIGHT,0,Alt account because friends know my real one. ...,alt account because friends know my real one. ...,"[alt, account, because, friends, know, my, rea...","[alt, account, because, friend, know, my, real...",alt account because friend know my real one . ...
5,Pretty much was hanging out with two friends. ...,WRONG,1,Pretty much was hanging out with two friends. ...,pretty much was hanging out with two friends. ...,"[pretty, much, was, hanging, out, with, two, f...","[pretty, much, wa, hanging, out, with, two, fr...",pretty much wa hanging out with two friend . i...
...,...,...,...,...,...,...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG,1,For reference we're both juniors at a T20 coll...,for reference we are both juniors at a t20 col...,"[for, reference, we, are, both, juniors, at, a...","[for, reference, we, are, both, junior, at, a,...",for reference we are both junior at a t20 coll...
2495,So this just happened and I'm a little taken a...,WRONG,1,So this just happened and I'm a little taken a...,so this just happened and i am a little taken ...,"[so, this, just, happened, and, i, am, a, litt...","[so, this, just, happened, and, i, am, a, litt...",so this just happened and i am a little taken ...
2497,"So, I've been friends with someone for a few y...",RIGHT,0,"So, I've been friends with someone for a few y...","so, i have been friends with someone for a few...","[so, ,, i, have, been, friends, with, someone,...","[so, ,, i, have, been, friend, with, someone, ...","so , i have been friend with someone for a few..."
2498,My first post!\n\nNo words were exchanged in t...,RIGHT,0,My first post! No words were exchanged in thi...,my first post! no words were exchanged in thi...,"[my, first, post, !, no, words, were, exchange...","[my, first, post, !, no, word, were, exchanged...",my first post ! no word were exchanged in this...


In [37]:
train_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1
2901,I work remotely while also studying in college...,RIGHT,0,I work remotely while also studying in college...,i work remotely while also studying in college...,"[i, work, remotely, while, also, studying, in,...","[i, work, remotely, while, also, studying, in,...",i work remotely while also studying in college...
10635,"I (29) am severely visually impaired, to the p...",WRONG,1,"I (29) am severely visually impaired, to the p...","i (29) am severely visually impaired, to the p...","[i, (, 29, ), am, severely, visually, impaired...","[i, (, 29, ), am, severely, visually, impaired...","i ( 29 ) am severely visually impaired , to th..."
3256,For the past 6 months my dad has been really c...,RIGHT,0,For the past 6 months my dad has been really c...,for the past 6 months my dad has been really c...,"[for, the, past, 6, months, my, dad, has, been...","[for, the, past, 6, month, my, dad, ha, been, ...",for the past 6 month my dad ha been really con...
3131,A little background. My wife and I are of diff...,RIGHT,0,A little background. My wife and I are of diff...,a little background. my wife and i are of diff...,"[a, little, background, ., my, wife, and, i, a...","[a, little, background, ., my, wife, and, i, a...",a little background . my wife and i are of dif...
4198,To sum up the entire story as quickly as possi...,RIGHT,0,To sum up the entire story as quickly as possi...,to sum up the entire story as quickly as possi...,"[to, sum, up, the, entire, story, as, quickly,...","[to, sum, up, the, entire, story, a, quickly, ...",to sum up the entire story a quickly a possibl...
...,...,...,...,...,...,...,...,...
6613,"So, to clarify, my sister and i have a fairly ...",WRONG,1,"So, to clarify, my sister and i have a fairly ...","so, to clarify, my sister and i have a fairly ...","[so, ,, to, clarify, ,, my, sister, and, i, ha...","[so, ,, to, clarify, ,, my, sister, and, i, ha...","so , to clarify , my sister and i have a fairl..."
7055,My ex girlfriend and I broke up at the beginni...,WRONG,1,My ex girlfriend and I broke up at the beginni...,my ex girlfriend and i broke up at the beginni...,"[my, ex, girlfriend, and, i, broke, up, at, th...","[my, ex, girlfriend, and, i, broke, up, at, th...",my ex girlfriend and i broke up at the beginni...
7200,So the title is a very condensed version of wh...,WRONG,1,So the title is a very condensed version of wh...,so the title is a very condensed version of wh...,"[so, the, title, is, a, very, condensed, versi...","[so, the, title, is, a, very, condensed, versi...",so the title is a very condensed version of wh...
200,"Several years ago when I was in college, my ol...",RIGHT,0,"Several years ago when I was in college, my ol...","several years ago when i was in college, my ol...","[several, years, ago, when, i, was, in, colleg...","[several, year, ago, when, i, wa, in, college,...","several year ago when i wa in college , my old..."


# **Testing story saperation**

In [38]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data = dev_df['clean'][0]
print('\n-----\n'.join(tokenizer.tokenize(data)))

My boyfriend (well just call him M) and I are creators on Instagram (he is also on TikTok).
-----
I'm a digital and traditional artist and in one of my classes we create t shirts and things (socks, lanyards, water bottles, stickers, you name it).
-----
I'm a big fan of putting my art on t shirts so I can sell them.
-----
M wanted to make some posts for TikTok and wanted to use the shirt I made today.
-----
My one personal rule is that if the shirt isn't being given to someone else, then I wear it first (this is because I know I wont get it back.
-----
Ive made one other shirt that I've worn, and he wore it for a week and hasn't given it back.)
-----
So I told him no, because I haven't worn the shirt yet.
-----
M proceeded to complain about not having content to post, and how his IG post today didn't get any engagement, saying "of course you dont know, because you never look."
-----
Am I the asshole for wanting to wear the shirt I made just once?


In [39]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data = dev_df['clean1'][0]
x=tokenizer.tokenize(data)
for i in x:
  print(i)
print(len(x))

my boyfriend ( well just call him m ) and i are creator on instagram ( he is also on tiktok ) .
i am a digital and traditional artist and in one of my class we create t shirt and thing ( sock , lanyard , water bottle , sticker , you name it ) .
i am a big fan of putting my art on t shirt so i can sell them .
m wanted to make some post for tiktok and wanted to use the shirt i made today .
my one personal rule is that if the shirt is not being given to someone else , then i wear it first ( this is because i know i will not get it back .
i have made one other shirt that i have worn , and he wore it for a week and ha not given it back . )
so i told him no , because i have not worn the shirt yet .
m proceeded to complain about not having content to post , and how his ig post today did not get any engagement , saying `` of course you do not know , because you never look . ''
am i the asshole for wanting to wear the shirt i made just once ?
9


In [40]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

train_sum = 0
num_of_sentences = []
# loop all the dataset, get text data and its label
train_line = []
train_line_label = []
for index, row in train_df.iterrows():
  sentence_count = 0 # for every new story, set the sentence counter to 0
  data = train_df['clean1'][index]
  label =  train_df['num_binarized_label'][index]
  saparated_text = tokenizer.tokenize(data)
  train_sum += len(saparated_text)
  for each_sentence in saparated_text: # loop through each saparated sentences.
    sentence_count += 1
    train_line.append(each_sentence) # add EACH SENTENCES to the list(line)
    train_line_label.append(label) # add its label to the list (line_label)
  num_of_sentences.append(sentence_count)
train_sentences = pd.DataFrame(train_line,columns=['line'])
train_sentences['line_label'] = train_line_label
if len(train_line) == train_sum:
  print('Training set has same len')
  train_df['num_of_sentences'] = num_of_sentences



dev_sum = 0
num_of_sentences = []
# loop all the dataset, get text data and its label
dev_line = []
dev_line_label = []
for index, row in dev_df.iterrows():
  sentence_count = 0 # for every new story, set the sentence counter to 0
  data = dev_df['clean1'][index]
  label =  dev_df['num_binarized_label'][index]
  saparated_text = tokenizer.tokenize(data)
  dev_sum += len(saparated_text)
  for each_sentence in saparated_text: # loop through each saparated sentences.
    sentence_count += 1
    dev_line.append(each_sentence) # add it to the list(line)
    dev_line_label.append(label) # add its label to the list (line_label)
  num_of_sentences.append(sentence_count)
dev_sentences = pd.DataFrame(dev_line,columns=['line'])
dev_sentences['line_label'] = dev_line_label
if len(dev_line) == dev_sum:
  print('DEV set has same len')
  dev_df['num_of_sentences'] = num_of_sentences



test_sum = 0
num_of_sentences = []
 # loop all the dataset, get text data and its label
test_line = []
test_line_label = []
for index, row in test_df.iterrows():
  sentence_count = 0 
  data = test_df['clean1'][index]
  label =  test_df['num_binarized_label'][index]
  saparated_text = tokenizer.tokenize(data)
  test_sum += len(saparated_text)
  for each_sentence in saparated_text: # loop through each saparated sentences.
    sentence_count += 1
    test_line.append(each_sentence) # add it to the list(line)
    test_line_label.append(label) # add its label to the list (line_label)
  num_of_sentences.append(sentence_count)
test_sentences = pd.DataFrame(test_line,columns=['line'])
test_sentences['line_label'] = test_line_label
if len(test_line) == test_sum:
  print('Test set has same len')
  test_df['num_of_sentences'] = num_of_sentences

Training set has same len
DEV set has same len
Test set has same len


In [41]:
train_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1,num_of_sentences
2901,I work remotely while also studying in college...,RIGHT,0,I work remotely while also studying in college...,i work remotely while also studying in college...,"[i, work, remotely, while, also, studying, in,...","[i, work, remotely, while, also, studying, in,...",i work remotely while also studying in college...,21
10635,"I (29) am severely visually impaired, to the p...",WRONG,1,"I (29) am severely visually impaired, to the p...","i (29) am severely visually impaired, to the p...","[i, (, 29, ), am, severely, visually, impaired...","[i, (, 29, ), am, severely, visually, impaired...","i ( 29 ) am severely visually impaired , to th...",32
3256,For the past 6 months my dad has been really c...,RIGHT,0,For the past 6 months my dad has been really c...,for the past 6 months my dad has been really c...,"[for, the, past, 6, months, my, dad, has, been...","[for, the, past, 6, month, my, dad, ha, been, ...",for the past 6 month my dad ha been really con...,3
3131,A little background. My wife and I are of diff...,RIGHT,0,A little background. My wife and I are of diff...,a little background. my wife and i are of diff...,"[a, little, background, ., my, wife, and, i, a...","[a, little, background, ., my, wife, and, i, a...",a little background . my wife and i are of dif...,29
4198,To sum up the entire story as quickly as possi...,RIGHT,0,To sum up the entire story as quickly as possi...,to sum up the entire story as quickly as possi...,"[to, sum, up, the, entire, story, as, quickly,...","[to, sum, up, the, entire, story, a, quickly, ...",to sum up the entire story a quickly a possibl...,21
...,...,...,...,...,...,...,...,...,...
6613,"So, to clarify, my sister and i have a fairly ...",WRONG,1,"So, to clarify, my sister and i have a fairly ...","so, to clarify, my sister and i have a fairly ...","[so, ,, to, clarify, ,, my, sister, and, i, ha...","[so, ,, to, clarify, ,, my, sister, and, i, ha...","so , to clarify , my sister and i have a fairl...",34
7055,My ex girlfriend and I broke up at the beginni...,WRONG,1,My ex girlfriend and I broke up at the beginni...,my ex girlfriend and i broke up at the beginni...,"[my, ex, girlfriend, and, i, broke, up, at, th...","[my, ex, girlfriend, and, i, broke, up, at, th...",my ex girlfriend and i broke up at the beginni...,11
7200,So the title is a very condensed version of wh...,WRONG,1,So the title is a very condensed version of wh...,so the title is a very condensed version of wh...,"[so, the, title, is, a, very, condensed, versi...","[so, the, title, is, a, very, condensed, versi...",so the title is a very condensed version of wh...,20
200,"Several years ago when I was in college, my ol...",RIGHT,0,"Several years ago when I was in college, my ol...","several years ago when i was in college, my ol...","[several, years, ago, when, i, was, in, colleg...","[several, year, ago, when, i, wa, in, college,...","several year ago when i wa in college , my old...",9


In [42]:
dev_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1,num_of_sentences
0,My boyfriend (well just call him M) and I are ...,RIGHT,0,My boyfriend (well just call him M) and I are ...,my boyfriend (well just call him m) and i are ...,"[my, boyfriend, (, well, just, call, him, m, )...","[my, boyfriend, (, well, just, call, him, m, )...",my boyfriend ( well just call him m ) and i ar...,9
2,\n\nMy really good friend is getting married....,RIGHT,0,My really good friend is getting married. Y...,my really good friend is getting married. y...,"[my, really, good, friend, is, getting, marrie...","[my, really, good, friend, is, getting, marrie...",my really good friend is getting married . yay...,25
3,"I've been very lonely for a long time, few fri...",RIGHT,0,"I've been very lonely for a long time, few fri...","i have been very lonely for a long time, few f...","[i, have, been, very, lonely, for, a, long, ti...","[i, have, been, very, lonely, for, a, long, ti...","i have been very lonely for a long time , few ...",6
4,Alt account because friends know my real one. ...,RIGHT,0,Alt account because friends know my real one. ...,alt account because friends know my real one. ...,"[alt, account, because, friends, know, my, rea...","[alt, account, because, friend, know, my, real...",alt account because friend know my real one . ...,18
5,Pretty much was hanging out with two friends. ...,WRONG,1,Pretty much was hanging out with two friends. ...,pretty much was hanging out with two friends. ...,"[pretty, much, was, hanging, out, with, two, f...","[pretty, much, wa, hanging, out, with, two, fr...",pretty much wa hanging out with two friend . i...,8
...,...,...,...,...,...,...,...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG,1,For reference we're both juniors at a T20 coll...,for reference we are both juniors at a t20 col...,"[for, reference, we, are, both, juniors, at, a...","[for, reference, we, are, both, junior, at, a,...",for reference we are both junior at a t20 coll...,25
2495,So this just happened and I'm a little taken a...,WRONG,1,So this just happened and I'm a little taken a...,so this just happened and i am a little taken ...,"[so, this, just, happened, and, i, am, a, litt...","[so, this, just, happened, and, i, am, a, litt...",so this just happened and i am a little taken ...,13
2497,"So, I've been friends with someone for a few y...",RIGHT,0,"So, I've been friends with someone for a few y...","so, i have been friends with someone for a few...","[so, ,, i, have, been, friends, with, someone,...","[so, ,, i, have, been, friend, with, someone, ...","so , i have been friend with someone for a few...",37
2498,My first post!\n\nNo words were exchanged in t...,RIGHT,0,My first post! No words were exchanged in thi...,my first post! no words were exchanged in thi...,"[my, first, post, !, no, words, were, exchange...","[my, first, post, !, no, word, were, exchanged...",my first post ! no word were exchanged in this...,25


In [43]:
train_sentences

,line,line_label
0,i work remotely while also studying in college...,0
1,so i know i have to be twice a good a everyone...,0
2,i do not earn a lot but i do pay for most of m...,0
3,they also owe me a decent bit of money - about...,0
4,but i do not bring it up because i do not real...,0
...,...,...
198620,"some of them speak english , but very often , ...",1
198621,"i am stuck there , sitting next to her , barel...",1
198622,when i expressed frustration with that circums...,1
198623,so ... am i the asshole for trying to get the ...,1


In [44]:
dev_sentences

,line,line_label
0,my boyfriend ( well just call him m ) and i ar...,0
1,i am a digital and traditional artist and in o...,0
2,i am a big fan of putting my art on t shirt so...,0
3,m wanted to make some post for tiktok and want...,0
4,my one personal rule is that if the shirt is n...,0
...,...,...
40891,after this incident we ( my suite mate and i )...,1
40892,he had broken my trust in a way he can not reg...,1
40893,we explained to her that it looked creepy and ...,1
40894,now my roommate seems annoyed that i do not wa...,1


In [45]:
test_sentences

,line,line_label
0,throwaway for obvious reason .,1
1,i dropped out of uni 2 year ago because i wa p...,1
2,"however , my parent did not want to help me in...",1
3,they have always supported my younger sister t...,1
4,"so atm i am stuck at home with them , which su...",1
...,...,...
41130,"the only class we share are math and form , wh...",0
41131,i spend the whole of lunch with her on wednesd...,0
41132,but i just hate her feeling left out when i am...,0
41133,"so , am i the asshole in this situation ?",0


# **Setup train test dataset**

In [46]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', lowercase=True)
'''text_transformer = CountVectorizer(input='content',
            encoding='utf-8',
            decode_error='strict',
            preprocessor=None,
            tokenizer=None,
            token_pattern=r'(?u)\b\w\w+\b',
            max_features=None,
            vocabulary=None,
            binary=True)'''


"text_transformer = CountVectorizer(input='content',\n            encoding='utf-8',\n            decode_error='strict',\n            preprocessor=None,\n            tokenizer=None,\n            token_pattern=r'(?u)\x08\\w\\w+\x08',\n            max_features=None,\n            vocabulary=None,\n            binary=True)"

In [47]:
train_x = train_sentences[["line"]]
train_y = train_sentences[["line_label"]]

dev_x = dev_sentences[["line"]]
dev_y = dev_sentences[["line_label"]]

test_x = test_sentences[["line"]]
test_y = test_sentences[["line_label"]]

In [48]:
train_x_text = text_transformer.fit_transform(train_x['line'])
dev_x_text = text_transformer.transform(dev_x['line'])
test_x_text = text_transformer.transform(test_x['line'])

In [49]:
train_x_text.shape

(198625, 30320)

# **GaussianNB**

In [50]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB

#https://stackoverflow.com/questions/39828535/how-to-tune-gaussiannb
model = GaussianNB()
model.fit(train_x_text.toarray(), train_y)

'''model = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=model, 
                 param_grid=params_NB, 
                 cv=5,   # use any cross validation technique 
                 verbose=1, 
                 scoring='accuracy')

gs_NB.fit(X, train_y)

gs_NB.best_params_
'''

"model = GaussianNB()\n\nparams_NB = {'var_smoothing': np.logspace(0,-9, num=100)}\ngs_NB = GridSearchCV(estimator=model, \n                 param_grid=params_NB, \n                 cv=5,   # use any cross validation technique \n                 verbose=1, \n                 scoring='accuracy')\n\ngs_NB.fit(X, train_y)\n\ngs_NB.best_params_\n"

In [53]:
test_pred = model.predict(test_x_text.toarray())

print(classification_report(test_y,test_pred))
print(accuracy_score(test_y, test_pred))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84     32488
           1       0.24      0.12      0.16      8647

    accuracy                           0.74     41135
   macro avg       0.51      0.51      0.50     41135
weighted avg       0.68      0.74      0.70     41135

0.7350674607998056


In [54]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 20:30:54


# **Vote for GaussianNB**

In [55]:
# get the both true and predicted labels as lists
test_true_labels = test_y
test_pred_labels = test_pred.flatten()


#get the number of sentences of everystory in every in all datasets
#train_sentence_count = train_df['num_of_sentences'].to_numpy().tolist()
#dev_sentence_count = dev_df['num_of_sentences'].to_numpy().tolist()
test_sentence_count = test_df['num_of_sentences'] .to_numpy().tolist()



print(type(test_sentence_count))
#print(test_sentence_count)
sentence_label_copy = np.array(test_pred_labels)



<class 'list'>


In [56]:
sentence_label_copy.shape

(41135,)

In [57]:
test_list_for_vote = [] # this list contains sublists of labels of sentences for every story

for i in test_sentence_count: # loop through each sentence counts(number of sentences for each story
  sub = [] # empty list for storing all the labels that belongs to that ONE particular story
  sub.append(sentence_label_copy[:i].tolist()) 
  # select the sentences labels that belong to the corresponding story and adding to sub

  index = [x for x in range(0,i)] # the index for selection and deletion
  sentence_label_copy = np.delete(sentence_label_copy, index) # delete the existed labels

  test_list_for_vote.append(sub[0]) # add the labels(for each story) to the list for later voting



def most_common(lst):
    return max(set(lst), key=lst.count)



# double checking and vote the most commom's label for each story
test_vote_result = [] 
for i in test_list_for_vote:
  if len(i) != 0:
    test_vote_result.append(most_common(i))
  else:
    test_vote_result.append(0)

In [58]:
# a copy of label of story
tbl =  test_df['num_binarized_label'] 
test_binarized_label =[]
for i in tbl:
  test_binarized_label.append(i)



# check the length is the same or not
print(len(test_binarized_label))
print(len(test_vote_result))



# final result
test_correct = 0
for i in range(0, 2358):
  if test_binarized_label[i] == test_vote_result[i]:
    test_correct += 1
print('accuracy:' , test_correct/2358)

2358
2358
accuracy: 0.7765055131467345


In [59]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
print(classification_report(test_binarized_label, test_vote_result))
print(confusion_matrix(test_binarized_label, test_vote_result))

              precision    recall  f1-score   support

           0       0.78      1.00      0.87      1827
           1       0.64      0.02      0.03       531

    accuracy                           0.78      2358
   macro avg       0.71      0.51      0.45      2358
weighted avg       0.75      0.78      0.68      2358

[[1822    5]
 [ 522    9]]


In [60]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 20:31:22


# **MultinomialNB**

In [61]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(train_x_text.toarray(), train_y)



MultinomialNB()

In [64]:
test_pred = model.predict(test_x_text.toarray())

print(confusion_matrix(test_y,test_pred))
print(classification_report(test_y,test_pred))
print(accuracy_score(test_y, test_pred))

[[20771 11717]
 [ 4838  3809]]
              precision    recall  f1-score   support

           0       0.81      0.64      0.72     32488
           1       0.25      0.44      0.32      8647

    accuracy                           0.60     41135
   macro avg       0.53      0.54      0.52     41135
weighted avg       0.69      0.60      0.63     41135

0.5975446699890604


# **Vote for MultinomialNB**

In [65]:
# get the both true and predicted labels as lists
test_true_labels = test_y
test_pred_labels = test_pred.flatten()


#get the number of sentences of everystory in every in all datasets
#train_sentence_count = train_df['num_of_sentences'].to_numpy().tolist()
#dev_sentence_count = dev_df['num_of_sentences'].to_numpy().tolist()
test_sentence_count = test_df['num_of_sentences'] .to_numpy().tolist()



print(type(test_sentence_count))
#print(test_sentence_count)
sentence_label_copy = np.array(test_pred_labels)



<class 'list'>


In [66]:
sentence_label_copy.shape

(41135,)

In [67]:
test_list_for_vote = [] # this list contains sublists of labels of sentences for every story

for i in test_sentence_count: # loop through each sentence counts(number of sentences for each story
  sub = [] # empty list for storing all the labels that belongs to that ONE particular story
  sub.append(sentence_label_copy[:i].tolist()) 
  # select the sentences labels that belong to the corresponding story and adding to sub

  index = [x for x in range(0,i)] # the index for selection and deletion
  sentence_label_copy = np.delete(sentence_label_copy, index) # delete the existed labels

  test_list_for_vote.append(sub[0]) # add the labels(for each story) to the list for later voting



def most_common(lst):
    return max(set(lst), key=lst.count)



# double checking and vote the most commom's label for each story
test_vote_result = [] 
for i in test_list_for_vote:
  if len(i) != 0:
    test_vote_result.append(most_common(i))
  else:
    test_vote_result.append(0)

In [68]:
# a copy of label of story
tbl =  test_df['num_binarized_label'] 
test_binarized_label =[]
for i in tbl:
  test_binarized_label.append(i)



# check the length is the same or not
print(len(test_binarized_label))
print(len(test_vote_result))



# final result
test_correct = 0
for i in range(0, 2358):
  if test_binarized_label[i] == test_vote_result[i]:
    test_correct += 1
print('accuracy:' , test_correct/2358)

2358
2358
accuracy: 0.6870229007633588


In [69]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
print(classification_report(test_binarized_label, test_vote_result))
print(confusion_matrix(test_binarized_label, test_vote_result))

              precision    recall  f1-score   support

           0       0.81      0.78      0.79      1827
           1       0.32      0.36      0.34       531

    accuracy                           0.69      2358
   macro avg       0.57      0.57      0.57      2358
weighted avg       0.70      0.69      0.69      2358

[[1428  399]
 [ 339  192]]


In [70]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 20:32:56
